In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
from matplotlib.pyplot import figure
import scipy as sp
import calendar
import seaborn as sns
import os
import gc
from scipy.stats import pearsonr
from stargazer.stargazer import Stargazer
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from datetime import datetime, date, timedelta

In [3]:
#!pip install tabulate

In [4]:
#!pip install stargazer

## Digital Fundametals

In [5]:
factor_dict=pd.read_excel('C:/Users/19084/My Backup Files/Digital Assets/dict_hat.xlsx')
factor_dict.head()

,Factor_hat,Category,FactorNum
0,AdrActCnt_hat,Activity,F1
1,BlkCnt_hat,Blocks,F2
2,BlkSizeByte_hat,Blocks,F3
3,BlkSizeMeanByte_hat,Blocks,F4
4,CapMrktCurUSD_hat,MarketCap,F5


# Pre-Process Data Functions

In [6]:
def factor_read(curr):
    wkdir = 'C:/Users/19084/My Backup Files/Digital Assets/'
    factor_data = wkdir + curr + '.csv'
    dfname = 'factor_data'
    dfname = pd.read_csv(factor_data)
    return dfname

In [7]:
def factor_df(df_in,curr):
    df_out=pd.DataFrame(columns=['Time'])
    df_out['Time']=df_in['Time']   
    for col in df_in.columns:
        for j in range(0,len(factor_dict)):
            if col==factor_dict['Factor_hat'].iloc[j]:
                new_col=curr + '_' + factor_dict['FactorNum'].iloc[j]
                df_out[new_col]=df_in[col]
    return df_out           

## Read and process data

In [8]:
btc_factor_05=factor_read('BTC/BTC_2022_05min01-06')
btc_factor_05=btc_factor_05.drop('Unnamed: 0',1)
btc_factor_30=factor_read('BTC/BTC_2022_30min01-06')
btc_factor_30=btc_factor_30.drop('Unnamed: 0',1)
btc_factor_60=factor_read('BTC/BTC_2022_60min01-06')
btc_factor_60=btc_factor_60.drop('Unnamed: 0',1)
eth_factor_05=factor_read('ETH/ETH_2022_05min01-06')
eth_factor_05=eth_factor_05.drop('Unnamed: 0',1)
eth_factor_30=factor_read('ETH/ETH_2022_30min01-06')
eth_factor_30=eth_factor_30.drop('Unnamed: 0',1)
eth_factor_60=factor_read('ETH/ETH_2022_60min01-06')
eth_factor_60=eth_factor_60.drop('Unnamed: 0',1)
btc_factor_05.head()

C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.
C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except fo

,Time,AdrActCnt_hat,BlkCnt_hat,BlkSizeMeanByte_hat,DiffMean_hat,IssContNtv_hat,SplyCur_hat,TxCnt_hat,HashRate_hat
0,1/1/2022 0:00,8749,1,1426751.0,2.427230e+13,6.25,18745187.54,962,347496240.4
1,1/1/2022 0:05,0,0,NaN,NaN,0.00,18745187.54,0,NaN
2,1/1/2022 0:10,9531,1,1496543.0,2.427230e+13,6.25,18745193.79,1466,347496240.4
3,1/1/2022 0:15,0,0,NaN,NaN,0.00,18745193.79,0,NaN
4,1/1/2022 0:20,8857,2,538075.0,2.427230e+13,12.50,18745206.29,1583,694992480.8


In [9]:
btc_factor_df_05=factor_df(btc_factor_05,'BTC')
eth_factor_df_05=factor_df(eth_factor_05,'ETH')
btc_factor_df_30=factor_df(btc_factor_30,'BTC')
eth_factor_df_30=factor_df(eth_factor_30,'ETH')
btc_factor_df_60=factor_df(btc_factor_60,'BTC')
eth_factor_df_60=factor_df(eth_factor_60,'ETH')
btc_factor_df_05.head()

,Time,BTC_F1,BTC_F2,BTC_F4,BTC_F6,BTC_F14,BTC_F20,BTC_F21,BTC_F13
0,1/1/2022 0:00,8749,1,1426751.0,2.427230e+13,6.25,18745187.54,962,347496240.4
1,1/1/2022 0:05,0,0,NaN,NaN,0.00,18745187.54,0,NaN
2,1/1/2022 0:10,9531,1,1496543.0,2.427230e+13,6.25,18745193.79,1466,347496240.4
3,1/1/2022 0:15,0,0,NaN,NaN,0.00,18745193.79,0,NaN
4,1/1/2022 0:20,8857,2,538075.0,2.427230e+13,12.50,18745206.29,1583,694992480.8


In [48]:
pd.to_datetime(btc_factor_df_05['Time'])
pd.to_datetime(eth_factor_df_05['Time'])
pd.to_datetime(btc_factor_df_30['Time'])
pd.to_datetime(eth_factor_df_30['Time'])
pd.to_datetime(btc_factor_df_60['Time'])
pd.to_datetime(eth_factor_df_60['Time'])

0      2022-01-01 00:00:00
1      2022-01-01 01:00:00
2      2022-01-01 02:00:00
3      2022-01-01 03:00:00
4      2022-01-01 04:00:00
               ...        
4339   2022-06-30 19:00:00
4340   2022-06-30 20:00:00
4341   2022-06-30 21:00:00
4342   2022-06-30 22:00:00
4343   2022-06-30 23:00:00
Name: Time, Length: 4344, dtype: datetime64[ns]

In [52]:
btc_eth_factor_df_05 = pd.merge(btc_factor_df_05,eth_factor_df_05, on='Time', how='outer')
btc_eth_factor_df_05.head()

,Time,BTC_F1,BTC_F2,BTC_F4,BTC_F6,BTC_F14,BTC_F20,BTC_F21,BTC_F13,ETH_F1,ETH_F2,ETH_F4,ETH_F6,ETH_F14,ETH_F20,ETH_F21,ETH_F22,ETH_F13
0,1/1/2022 0:00,8749.0,1.0,1426751.0,2.427230e+13,6.25,18745187.54,962.0,347496240.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/1/2022 0:05,0.0,0.0,NaN,NaN,0.00,18745187.54,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1/1/2022 0:10,9531.0,1.0,1496543.0,2.427230e+13,6.25,18745193.79,1466.0,347496240.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1/1/2022 0:15,0.0,0.0,NaN,NaN,0.00,18745193.79,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/1/2022 0:20,8857.0,2.0,538075.0,2.427230e+13,12.50,18745206.29,1583.0,694992480.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merge BTC-ETH

In [50]:
btc_eth_factor_df_05 = pd.merge(btc_factor_df_05,eth_factor_df_05, on='Time', how='outer')
btc_eth_factor_df_30 = pd.merge(btc_factor_df_30,eth_factor_df_30, on='Time', how='outer')
btc_eth_factor_df_60 = pd.merge(btc_factor_df_60,eth_factor_df_60, on='Time', how='outer')
btc_eth_factor_clean_df_05=btc_eth_factor_df_05.dropna().copy()
btc_eth_factor_clean_df_30=btc_eth_factor_df_30.dropna().copy()
btc_eth_factor_clean_df_60=btc_eth_factor_df_60.dropna().copy()
btc_eth_factor_clean_df_05.head()

,Time,BTC_F1,BTC_F2,BTC_F4,BTC_F6,BTC_F14,BTC_F20,BTC_F21,BTC_F13,ETH_F1,ETH_F2,ETH_F4,ETH_F6,ETH_F14,ETH_F20,ETH_F21,ETH_F22,ETH_F13


# Prices

In [11]:
def price_returns(curr,t):
    wkdir = 'C:/Users/19084/My Backup Files/Digital Assets/'
    year = '2022'
    price_data = wkdir+ curr + '/' + curr + 'USD_' + year + '_' + t + 'min.csv'
    temp = pd.read_csv(price_data)
    #curr_price = curr+'_USD'
    df_out=pd.DataFrame(columns=['Time',curr +'_USD',curr +'_Ret'])
    df_out['Time']=temp['Date'] #'Date_Time'
    df_out[curr+'_USD']=temp['Close']
    df_out[curr+'_Ret']=temp['pct']
    return df_out
    # return df_out

In [12]:
btc_price_05=price_returns('BTC','5')
eth_price_05=price_returns('ETH','5')
btc_price_30=price_returns('BTC','30')
eth_price_30=price_returns('ETH','30')
btc_price_60=price_returns('BTC','60')
eth_price_60=price_returns('ETH','60')
btc_price_05.head()

,Time,BTC_USD,BTC_Ret
0,2022-01-01 00:00:00,46224.93,0.000000
1,2022-01-01 00:05:00,46411.40,0.004034
2,2022-01-01 00:10:00,46368.76,-0.000919
3,2022-01-01 00:15:00,46316.96,-0.001117
4,2022-01-01 00:20:00,46290.21,-0.000578


## Merge BTC-ETH prices and returns

In [13]:
btc_eth_price_05=pd.merge(btc_price_05,eth_price_05, on='Time', how='outer')
btc_eth_price_30=pd.merge(btc_price_30,eth_price_30, on='Time', how='outer')
btc_eth_price_60=pd.merge(btc_price_60,eth_price_60, on='Time', how='outer')
#btc_eth_price_clean=btc_eth_price.dropna()
btc_eth_price_05['BTC_ETH_Ret']=btc_eth_price_05['BTC_Ret']-btc_eth_price_05['ETH_Ret']
btc_eth_price_30['BTC_ETH_Ret']=btc_eth_price_30['BTC_Ret']-btc_eth_price_30['ETH_Ret']
btc_eth_price_60['BTC_ETH_Ret']=btc_eth_price_60['BTC_Ret']-btc_eth_price_60['ETH_Ret']
btc_eth_price_05.head()

,Time,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret
0,2022-01-01 00:00:00,46224.93,0.000000,3683.31,0.000000,0.000000
1,2022-01-01 00:05:00,46411.40,0.004034,3699.59,0.004420,-0.000386
2,2022-01-01 00:10:00,46368.76,-0.000919,3699.12,-0.000127,-0.000792
3,2022-01-01 00:15:00,46316.96,-0.001117,3691.26,-0.002125,0.001008
4,2022-01-01 00:20:00,46290.21,-0.000578,3690.85,-0.000111,-0.000466


In [14]:
btc_eth_price_05['Time']=btc_eth_price_05['Time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))  
btc_eth_price_30['Time']=btc_eth_price_30['Time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))  
btc_eth_price_60['Time']=btc_eth_price_60['Time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))  

In [15]:
btc_eth_factor_clean_df_05['Time']=btc_eth_factor_clean_df_05['Time'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M"))  
btc_eth_factor_clean_df_30['Time']=btc_eth_factor_clean_df_30['Time'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M"))  
btc_eth_factor_clean_df_60['Time']=btc_eth_factor_clean_df_60['Time'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M"))  

In [16]:
btc_eth_model_05=pd.merge(btc_eth_price_05,btc_eth_factor_clean_df_05, on='Time', how='outer')
btc_eth_model_30=pd.merge(btc_eth_price_30,btc_eth_factor_clean_df_30, on='Time', how='outer')
btc_eth_model_60=pd.merge(btc_eth_price_60,btc_eth_factor_clean_df_60, on='Time', how='outer')
btc_eth_model_05_keep=btc_eth_model_05.dropna()
btc_eth_model_30_keep=btc_eth_model_30.dropna()
btc_eth_model_60_keep=btc_eth_model_60.dropna()
btc_eth_model_05=btc_eth_model_05_keep
btc_eth_model_30=btc_eth_model_30_keep
btc_eth_model_60=btc_eth_model_60_keep

In [17]:
btc_eth_model_05.head()

,Time,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,BTC_F1,BTC_F2,BTC_F4,BTC_F6,...,BTC_F13,ETH_F1,ETH_F2,ETH_F4,ETH_F6,ETH_F14,ETH_F20,ETH_F21,ETH_F22,ETH_F13


In [18]:
c_btc_eth_model_05=btc_eth_model_05.drop(['BTC_USD','ETH_USD'], axis=1)
c_btc_eth_model_30=btc_eth_model_30.drop(['BTC_USD','ETH_USD'], axis=1)
c_btc_eth_model_60=btc_eth_model_60.drop(['BTC_USD','ETH_USD'], axis=1)

In [19]:
min05_correl = c_btc_eth_model_05.corr()
min30_correl = c_btc_eth_model_30.corr()
min60_correl = c_btc_eth_model_60.corr()

In [20]:
min05_correl.to_csv('05min_correl_matrix.csv',index=False)
min30_correl.to_csv('30min_correl_matrix.csv',index=False)
min60_correl.to_csv('60min_correl_matrix.csv',index=False)

In [21]:
btc_eth_model_05.corr()

,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,BTC_F1,BTC_F2,BTC_F4,BTC_F6,BTC_F14,...,BTC_F13,ETH_F1,ETH_F2,ETH_F4,ETH_F6,ETH_F14,ETH_F20,ETH_F21,ETH_F22,ETH_F13
BTC_USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC_Ret,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ETH_USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ETH_Ret,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC_ETH_Ret,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC_F1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC_F2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC_F4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC_F6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC_F14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Export Data

In [22]:
btc_eth_model_05.to_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_05_22.csv',index=False)
btc_eth_model_30.to_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_30_22.csv',index=False)
btc_eth_model_60.to_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_60_22.csv',index=False)

## Start Here

In [23]:
btc_eth_factor_clean_df_05=pd.read_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_05_22.csv')
btc_eth_factor_clean_df_30=pd.read_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_30_22.csv')
btc_eth_factor_clean_df_60=pd.read_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_60_22.csv')

# Modeling Returns

## Lag the factors

In [24]:
def relative_factors(df_in,first_col,curr1,curr2):
    df_out=df_in.iloc[:,0:first_col]
    for j in range(1,32):
        curr1_col=curr1 + '_F' + str(j)
        curr2_col=curr2 + '_F' + str(j)    
        new_col=curr1 + '_' + curr2 + '_F' + str(j) + '_ratio'
        try:
            df_out[new_col]=df_in[curr1_col]/df_in[curr2_col]
        except:
            continue
    return df_out

## Predictive Analysis

In [25]:
def factor_lag(df_in,first_col,last_col):
    df_out=df_in.iloc[:,0:first_col]
    j=first_col
    for j in range(first_col,last_col):
        col_name_old=df_in.columns[j]
        col_name_new='L_'+col_name_old
        df_out[col_name_new]=df_in.iloc[:,j].shift(1)
        j=j+1
    return df_out

In [26]:
shape = btc_eth_model_60.shape
print('Number of Columns : ', shape[1])

Number of Columns :  23


In [27]:
btc_eth_model_05_lags=factor_lag(btc_eth_model_05,6,23)
btc_eth_model_30_lags=factor_lag(btc_eth_model_30,6,23)
btc_eth_model_60_lags=factor_lag(btc_eth_model_60,6,23)
btc_eth_model_lags_05_keep=btc_eth_model_05_lags.dropna()
btc_eth_model_lags_30_keep=btc_eth_model_30_lags.dropna()
btc_eth_model_lags_60_keep=btc_eth_model_60_lags.dropna()
btc_eth_model_05_lags=btc_eth_model_lags_05_keep
btc_eth_model_30_lags=btc_eth_model_lags_30_keep
btc_eth_model_60_lags=btc_eth_model_lags_60_keep

In [28]:
btc_eth_model_05_lags.head()

,Time,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,L_BTC_F1,L_BTC_F2,L_BTC_F4,L_BTC_F6,...,L_BTC_F13,L_ETH_F1,L_ETH_F2,L_ETH_F4,L_ETH_F6,L_ETH_F14,L_ETH_F20,L_ETH_F21,L_ETH_F22,L_ETH_F13


## Create model string

In [29]:
lm_string_BTC='BTC_Ret ~ L_BTC_F1 + L_BTC_F2 + L_BTC_F4 + L_BTC_F6+ L_BTC_F14 + L_BTC_F20 + L_BTC_F21 + L_BTC_F13'
lm_string_ETH='ETH_Ret ~ L_ETH_F1 + L_ETH_F2 + L_ETH_F4 + L_ETH_F6+ L_ETH_F14 + L_ETH_F20 + L_ETH_F21 + L_ETH_F22 + L_ETH_F13'
print(lm_string_BTC)
print(lm_string_ETH)

BTC_Ret ~ L_BTC_F1 + L_BTC_F2 + L_BTC_F4 + L_BTC_F6+ L_BTC_F14 + L_BTC_F20 + L_BTC_F21 + L_BTC_F13
ETH_Ret ~ L_ETH_F1 + L_ETH_F2 + L_ETH_F4 + L_ETH_F6+ L_ETH_F14 + L_ETH_F20 + L_ETH_F21 + L_ETH_F22 + L_ETH_F13


In [30]:
lag_model_BTC_05=smf.ols(lm_string_BTC, data = btc_eth_model_05_lags).fit().summary()
print(lag_model_BTC_05)

ValueError: zero-size array to reduction operation maximum which has no identity

In [58]:
lag_model_BTC_30=smf.ols(lm_string_BTC, data = btc_eth_model_30_lags).fit().summary()
print(lag_model_BTC_30)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.924
Date:                Wed, 21 Sep 2022   Prob (F-statistic):             0.0731
Time:                        22:43:38   Log-Likelihood:                 32668.
No. Observations:                8424   AIC:                        -6.532e+04
Df Residuals:                    8417   BIC:                        -6.527e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.555e-10   2.26e-10     -1.129      0.2

In [59]:
lag_model_BTC_60=smf.ols(lm_string_BTC, data = btc_eth_model_60_lags).fit().summary()
print(lag_model_BTC_60)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.142
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.335
Time:                        22:43:41   Log-Likelihood:                 15585.
No. Observations:                4401   AIC:                        -3.116e+04
Df Residuals:                    4394   BIC:                        -3.111e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.513e-10   2.62e-10     -0.959      0.3

In [60]:
lag_model_ETH_05=smf.ols(lm_string_ETH, data = btc_eth_model_05_lags).fit().summary()
print(lag_model_ETH_05)

                            OLS Regression Results                            
Dep. Variable:                ETH_Ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.069
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.379
Time:                        22:43:42   Log-Likelihood:                 97076.
No. Observations:               21431   AIC:                        -1.941e+05
Df Residuals:                   21424   BIC:                        -1.941e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.024e-14   1.53e-14      1.974      0.0

In [61]:
lag_model_ETH_30=smf.ols(lm_string_ETH, data = btc_eth_model_30_lags).fit().summary()
print(lag_model_ETH_30)

                            OLS Regression Results                            
Dep. Variable:                ETH_Ret   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.343
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.234
Time:                        22:43:43   Log-Likelihood:                 30925.
No. Observations:                8424   AIC:                        -6.184e+04
Df Residuals:                    8417   BIC:                        -6.179e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.734e-12   5.55e-12      0.673      0.5

In [62]:
lag_model_ETH_60=smf.ols(lm_string_ETH, data = btc_eth_model_60_lags).fit().summary()
print(lag_model_ETH_60)

                            OLS Regression Results                            
Dep. Variable:                ETH_Ret   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.122
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.346
Time:                        22:43:47   Log-Likelihood:                 14604.
No. Observations:                4401   AIC:                        -2.919e+04
Df Residuals:                    4394   BIC:                        -2.915e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.286e-13   4.75e-13     -0.271      0.7

# Ratio Models

## Relative factors

In [63]:
def relative_factors(df_in,first_col,curr1,curr2):
    df_out=df_in.iloc[:,0:first_col]
    for j in range(1,32):
        curr1_col=curr1 + '_F' + str(j)
        curr2_col=curr2 + '_F' + str(j)   
        new_col=curr1 + '_' + curr2 + '_F' + str(j) + '_ratio'
        try:
            df_out[new_col]=df_in[curr1_col]/df_in[curr2_col]
        except:
            continue
    return df_out

In [64]:
btc_eth_factor_05_ratios=relative_factors(btc_eth_model_05,6,'BTC','ETH')
btc_eth_factor_30_ratios=relative_factors(btc_eth_model_30,6,'BTC','ETH')
btc_eth_factor_60_ratios=relative_factors(btc_eth_model_60,6,'BTC','ETH')
btc_eth_factor_05_ratios.head()

,Time,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,BTC_ETH_F1_ratio,BTC_ETH_F2_ratio,BTC_ETH_F4_ratio,BTC_ETH_F6_ratio,BTC_ETH_F13_ratio,BTC_ETH_F14_ratio,BTC_ETH_F20_ratio,BTC_ETH_F21_ratio
52131,2021-07-01 00:15:00,34824.02,-0.000889,2241.68,-0.006171,0.005282,3.553463,0.058824,25.423494,0.002992,7.362277e+05,0.174520,0.160891,1.241404
52134,2021-07-01 00:30:00,34825.91,0.003317,2246.00,0.002679,0.000639,4.162797,0.062500,25.738271,0.003009,8.529288e+05,0.195312,0.160891,0.866897
52135,2021-07-01 00:35:00,34814.48,-0.000328,2252.98,0.003108,-0.003436,3.684084,0.052632,27.068709,0.003016,6.957345e+05,0.156986,0.160891,0.696128
52138,2021-07-01 00:50:00,34846.75,0.000336,2251.78,0.000524,-0.000189,2.844329,0.038462,21.379917,0.003011,5.190004e+05,0.116144,0.160891,0.395305
52140,2021-07-01 01:00:00,34749.05,-0.003086,2249.41,-0.002912,-0.000174,4.306365,0.080000,25.806278,0.002999,1.058968e+06,0.241255,0.160891,0.317908


In [65]:
min05_correl_ratio = btc_eth_factor_05_ratios.corr()
min30_correl_ratio = btc_eth_factor_30_ratios.corr()
min60_correl_ratio = btc_eth_factor_60_ratios.corr()
min05_correl_ratio.to_csv('05min_ratio_correl_matrix.csv',index=False)
min30_correl_ratio.to_csv('30min_ratio_correl_matrix.csv',index=False)
min60_correl_ratio.to_csv('60min__ratio_correl_matrix.csv',index=False)

In [66]:
btc_eth_factor_05_ratios_lags=factor_lag(btc_eth_factor_05_ratios,6,14)
btc_eth_factor_30_ratios_lags=factor_lag(btc_eth_factor_30_ratios,6,14)
btc_eth_factor_60_ratios_lags=factor_lag(btc_eth_factor_60_ratios,6,14)
btc_eth_factor_05_ratios_lags_keep=btc_eth_factor_05_ratios_lags.dropna()
btc_eth_factor_30_ratios_lags_keep=btc_eth_factor_30_ratios_lags.dropna()
btc_eth_factor_60_ratios_lags_keep=btc_eth_factor_60_ratios_lags.dropna()
btc_eth_factor_05_ratios_lags=btc_eth_factor_05_ratios_lags_keep
btc_eth_factor_30_ratios_lags=btc_eth_factor_30_ratios_lags_keep
btc_eth_factor_60_ratios_lags=btc_eth_factor_60_ratios_lags_keep
btc_eth_factor_05_ratios_lags.head()

,Time,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,L_BTC_ETH_F1_ratio,L_BTC_ETH_F2_ratio,L_BTC_ETH_F4_ratio,L_BTC_ETH_F6_ratio,L_BTC_ETH_F13_ratio,L_BTC_ETH_F14_ratio,L_BTC_ETH_F20_ratio,L_BTC_ETH_F21_ratio
52134,2021-07-01 00:30:00,34825.91,0.003317,2246.00,0.002679,0.000639,3.553463,0.058824,25.423494,0.002992,7.362277e+05,0.174520,0.160891,1.241404
52135,2021-07-01 00:35:00,34814.48,-0.000328,2252.98,0.003108,-0.003436,4.162797,0.062500,25.738271,0.003009,8.529288e+05,0.195312,0.160891,0.866897
52138,2021-07-01 00:50:00,34846.75,0.000336,2251.78,0.000524,-0.000189,3.684084,0.052632,27.068709,0.003016,6.957345e+05,0.156986,0.160891,0.696128
52140,2021-07-01 01:00:00,34749.05,-0.003086,2249.41,-0.002912,-0.000174,2.844329,0.038462,21.379917,0.003011,5.190004e+05,0.116144,0.160891,0.395305
52142,2021-07-01 01:10:00,34835.68,0.000398,2254.65,-0.000545,0.000943,4.306365,0.080000,25.806278,0.002999,1.058968e+06,0.241255,0.160891,0.317908


In [67]:
lm_string_BTC_ETH_Ratio='BTC_Ret ~ L_BTC_ETH_F1_ratio + L_BTC_ETH_F2_ratio + L_BTC_ETH_F4_ratio + L_BTC_ETH_F6_ratio + L_BTC_ETH_F13_ratio + L_BTC_ETH_F14_ratio + L_BTC_ETH_F20_ratio + L_BTC_ETH_F21_ratio'
print(lm_string_BTC_ETH_Ratio)

BTC_Ret ~ L_BTC_ETH_F1_ratio + L_BTC_ETH_F2_ratio + L_BTC_ETH_F4_ratio + L_BTC_ETH_F6_ratio + L_BTC_ETH_F13_ratio + L_BTC_ETH_F14_ratio + L_BTC_ETH_F20_ratio + L_BTC_ETH_F21_ratio


In [68]:
ratio_model_05=smf.ols(lm_string_BTC_ETH_Ratio, data = btc_eth_factor_05_ratios_lags).fit().summary()
print(ratio_model_05)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.8965
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.518
Time:                        22:44:03   Log-Likelihood:             1.0168e+05
No. Observations:               21431   AIC:                        -2.033e+05
Df Residuals:                   21422   BIC:                        -2.033e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.0001    

In [69]:
ratio_model_30=smf.ols(lm_string_BTC_ETH_Ratio, data = btc_eth_factor_30_ratios_lags).fit().summary()
print(ratio_model_30)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9951
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.437
Time:                        22:44:10   Log-Likelihood:                 32666.
No. Observations:                8424   AIC:                        -6.531e+04
Df Residuals:                    8415   BIC:                        -6.525e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0286    

In [152]:
ratio_model_60=smf.ols(lm_string_BTC_ETH_Ratio, data = btc_eth_factor_60_ratios_lags).fit().summary()
print(ratio_model_60)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.284
Date:                Tue, 15 Mar 2022   Prob (F-statistic):              0.247
Time:                        00:12:20   Log-Likelihood:                 11791.
No. Observations:                3861   AIC:                        -2.356e+04
Df Residuals:                    3852   BIC:                        -2.351e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.5244    